# HW 3 Classification
28 Feb 2025



In [1]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-wongluyii')

### Landsat 8 Image Selection

In [3]:
# Modified Version
# Pulling Phildelphia boundary (Region of Interest)
philly = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq("NAME", "Philadelphia"))

# Load Landsat 8 Collection 
landsat = (ee.ImageCollection("LANDSAT/LC08/C02/T1")
           .filterBounds(philly)
           .filterDate("2023-01-01", "2023-12-31")
           .filterMetadata("CLOUD_COVER", "less_than", 10)
           .median()
           .clip(philly)
           .select("B[1-7]"))


# Visualize ROI and Landsat image
Map = geemap.Map()
Map.centerObject(philly, 10)

stats = landsat.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=philly,
    scale=30
)
min_value = stats.get("B4_min")
max_value = stats.get("B4_max")
max_false = stats.get("B7_max")
max_veg = stats.get("B5_max")
Map.addLayer(landsat, {"bands": ["B4", "B3", "B2"], "min": 5, "max": max_value}, "True Color")
Map.addLayer(landsat, {"bands": ["B7", "B4", "B6"], "min": 5, "max": max_false}, "False Color (Urban)")
Map.addLayer(landsat, {"bands": ["B5", "B4", "B3"], "min": 5, "max": max_veg}, "False Color (Veg)")
Map

Map(center=[40.00760439369441, -75.13403899470623], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Map.addLayer(landsat, {"bands": ["B5", "B4", "B3"], "min": 5, "max": max_veg}, "False Color (Veg)")
# Define a boundary
small_area = philly.geometry().buffer(10)
# Export False color (Veg)
geemap.ee_export_image(landsat,  scale=30, filename="landsat_philly_2023_v2.tif", region=small_area)

## Pull in Sample Data 

In [13]:
# Pull in manually labeled data from QGIS
training_samples = geemap.geojson_to_ee("/Users/luyiiwong/Documents/GitHub/musa-650-spring-2025/assignments/samples.geojson")

In [14]:
# Merge training samples based on class 
urban = training_samples.filter(ee.Filter.eq("class", "urban"))
bare = training_samples.filter(ee.Filter.eq("class", "bare"))
water = training_samples.filter(ee.Filter.eq("class", "water"))
vegetation = training_samples.filter(ee.Filter.eq("class", "vegetation"))

# Merge all labeled samples into a single dataset
training_data = urban.merge(bare).merge(water).merge(vegetation)

In [15]:
# Update classifiers into a numeric labels - create label mapping
label_map = ee.Dictionary({
    "urban": 0,
    "bare": 1,
    "water": 2,
    "vegetation": 3
})

# Function to assign numeric labels
def add_numeric_label(feature):
    return feature.set("landcover", label_map.get(feature.get("class")))

# Apply function to training data
training_data = training_data.map(add_numeric_label)


## Feature Engineering

The Normal Difference Vegetation Index (NDVI), Normal Difference Built-up Index (NDBI), and MNDWI (Modified Normalized Difference Water Index) can be calculated using the following:
- NDVI = (B5 - B4)/(B5 + B4)
- NDBI = (B6 - B5)/(B6 + B5)
- MNDWI = (B3 - B6)/(B3 + B6)

In [16]:
# Use normalizedDifference to compute (first − second) / (first + second)
# Calculate NDVI
ndvi = landsat.normalizedDifference(["B5", "B4"]).rename("NDVI")

# Calculate NDBI
ndbi = landsat.normalizedDifference(["B6", "B5"]).rename("NDBI")

# Calculate MNDWI
mndwi = landsat.normalizedDifference(["B3", "B6"]).rename("MNDWI")

In [17]:
# Load and normalize elevation data
dem = ee.Image("USGS/SRTMGL1_003").clip(philly)
slope = ee.Terrain.slope(dem)

# Normalize bands to 0-1 scale
def normalize(image, band):
    min_max = image.reduceRegion(ee.Reducer.minMax(), philly, 30)
    min_val = ee.Number(min_max.get(f"{band}_min"))
    max_val = ee.Number(min_max.get(f"{band}_max"))
    return image.select(band).subtract(min_val).divide(max_val.subtract(min_val)).rename(f"{band}_norm")

bands_to_normalize = ["B1", "B2", "B3", "B4", "B5", "B6", "B7"]
normalized_bands = [normalize(landsat, band) for band in bands_to_normalize]

# Combine all features
final_image = landsat.addBands([ndvi, ndbi, mndwi, slope] + normalized_bands)

In [21]:
# Visualize 
Map = geemap.Map()
Map.centerObject(philly, 10)

# Define visualization parameters for indices
ndvi_vis = {"min": -1, "max": 1, "palette": ["blue", "white", "green"]}
ndbi_vis = {"min": -1, "max": 1, "palette": ["blue", "white", "red"]}
mndwi_vis = {"min": -1, "max": 1, "palette": ["brown", "white", "blue"]}
slope_vis = {"min": 0, "max": 60, "palette": ["white", "yellow", "red"]}

# Add layers to the map
Map.addLayer(landsat, {"bands": ["B4", "B3", "B2"], "min": 5, "max": max_value}, "Landsat 8 (True Color)")
Map.addLayer(ndvi, ndvi_vis, "NDVI")
Map.addLayer(ndbi, ndbi_vis, "NDBI")
Map.addLayer(mndwi, mndwi_vis, "MNDWI")
Map.addLayer(slope, slope_vis, "Slope")

# Display the map
Map

EEException: Image.visualize: No band named 'B5'. Available band names: [NDVI].

In [22]:

# Initialize the map
Map = geemap.Map()
Map.centerObject(philly, 10)

# Create an RGB composite for each index using itself as R, G, and B
ndvi_rgb = ndvi.visualize(**{"min": -1, "max": 1, "palette": ["blue", "white", "green"]})
ndbi_rgb = ndbi.visualize(**{"min": -1, "max": 1, "palette": ["blue", "white", "red"]})
mndwi_rgb = mndwi.visualize(**{"min": -1, "max": 1, "palette": ["brown", "white", "blue"]})
slope_rgb = slope.visualize(**{"min": 0, "max": 60, "palette": ["white", "yellow", "red"]})

# Add layers to the map
Map.addLayer(landsat, {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000}, "Landsat 8 (True Color)")
Map.addLayer(ndvi_rgb, {}, "NDVI (False Color)")
Map.addLayer(ndbi_rgb, {}, "NDBI (False Color)")
Map.addLayer(mndwi_rgb, {}, "MNDWI (False Color)")
Map.addLayer(slope_rgb, {}, "Slope (False Color)")

# Display the map
Map


Map(center=[40.00760439369441, -75.13403899470623], controls=(WidgetControl(options=['position', 'transparent_…

In [70]:
# Landsat 8 Level 2, Collection 2, Tier 1
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
.filterDate('2021-05-01', '2021-06-01')

# Applies scaling factors.
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True)

dataset = dataset.map(applyScaleFactors)

vis_params = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'gamma': 1.0, 'min': 0.0, 'max': 0.3}

Map.setCenter(-75.1652, 39.9526, 10) 

Map.addLayer(dataset, vis_params, 'Landsat8')

In [71]:
Map

Map(bottom=50087.0, center=[39.65011210186374, -74.70153808593751], controls=(WidgetControl(options=['position…

In [60]:
# NDVI Landsat 8 Collection 2 Tier 1 Level 2 8-day composite
NDVI = ee.ImageCollection('LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDVI')
Map.addLayer(NDVI, {}, 'LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDVI')

# NDWI 
NDWI = ee.ImageCollection('LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDWI')
Map.addLayer(NDWI, {}, 'LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDWI')

In [62]:
# Downloading Google Earth Engine Catalog 
from geemap.datasets import DATA

### Check Image Properties

In [18]:
print("The Image Date is:", ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo())
print("The cloud cover is:", image.get("CLOUD_COVER").getInfo())

The Image Date is: 2023-04-10
The cloud cover is: 0.08


### Preparing Training Dataset

In [19]:
# Creating a buffer 
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
region = ee.Geometry.Point([-75.1652, 39.9526]).buffer(10000)


In [29]:
# NLCD Classification
nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(image.geometry())
Map.addLayer(nlcd, {}, "NLCD")
Map

# Make the training dataset.
points = nlcd.sample(
    **{
        "region": image.geometry(),
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)

In [31]:
# Training the Classifier
# Use these bands for prediction.
bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]


# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'SR_B1': 8130, 'SR_B2': 8129, 'SR_B3': 8496, 'SR_B4': 8544, 'SR_B5': 10505, 'SR_B6': 12037, 'SR_B7': 10508, 'landcover': 42}}


In [32]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")
Map

Map(center=[39.9526, -75.1652], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…